In [ ]:
import os
import cv2
import numpy as np

# Function to display the image and allow manual rectangle selection
def select_rectangle(image_path):
    image = cv2.imread(image_path)
    
    # Create a copy of the image to draw on
    draw_image = image.copy()
    
    # Variables to track mouse events
    drawing = False
    rectangle = None
    
    # Callback function for mouse event
    def on_mouse_event(event, x, y, flags, param):
        nonlocal drawing, rectangle, draw_image
        if event == cv2.EVENT_LBUTTONDOWN:
            drawing = True
            rectangle = (x, y, 0, 0)
        elif event == cv2.EVENT_LBUTTONUP:
            drawing = False
        elif event == cv2.EVENT_MOUSEMOVE:
            if drawing:
                rectangle = (rectangle[0], rectangle[1], x - rectangle[0], y - rectangle[1])
    
    # Create a window to display the image
    cv2.namedWindow("Select Rectangle")
    cv2.setMouseCallback("Select Rectangle", on_mouse_event)
    
    while True:
        # Display the current state of the selection
        draw_image = image.copy()
        if rectangle is not None:
            cv2.rectangle(draw_image, (rectangle[0], rectangle[1]), (rectangle[0] + rectangle[2], rectangle[1] + rectangle[3]), (0, 255, 0), 2)
        cv2.imshow("Select Rectangle", draw_image)
        
        # Wait for a key press (0 ms delay to enable window events)
        key = cv2.waitKey(1) & 0xFF
        
        if key == ord('c'):  # Press 'c' to clear the selection
            rectangle = None
        elif key == 27:  # Press 'Esc' to finish selection and move to the next image
            break
        elif key == 13:  # Press 'Enter' to save the selected rectangle as a mask and move to the next image
            if rectangle is not None:
                # Convert the rectangle to a binary mask
                mask = np.zeros(image.shape[:2], dtype=np.uint8)
                x, y, w, h = rectangle
                mask[y:y+h, x:x+w] = 1
                
                # Save the binary mask to the "masks" folder
                filename = os.path.basename(image_path)
                mask_filename = os.path.splitext(filename)[0] + "_mask.png"
                mask_path = os.path.join("masks", mask_filename)
                cv2.imwrite(mask_path, mask * 255)
                
            # Clear the selection for the next image
            rectangle = None
            
            # Move to the next image
            break
    
    # Close the window
    cv2.destroyWindow("Select Rectangle")

# Specify the directory where your images are stored
images_dir = "img/not_labeled_stones/"

# Create the "masks" folder to save the binary masks
os.makedirs("masks", exist_ok=True)

# List all image files in the directory
image_files = os.listdir(images_dir)

# Loop through each image and allow manual rectangle selection
for image_file in image_files:
    image_path = os.path.join(images_dir, image_file)
    print(f"Selecting rectangle for: {image_file}")
    select_rectangle(image_path)

print("Rectangle selection completed.")